# HYDRAD Configuration Example

In [ ]:
import copy
import pathlib

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
from sunpy.coordinates import get_horizons_coord

import synthesizAR
from synthesizAR.models import semi_circular_arcade, semi_circular_bundle
from synthesizAR.interfaces.hydrad import HYDRADInterface

import pydrad.configure
import pydrad.configure.data

## Set up strand geometry

In [ ]:
@u.quantity_input
def mikic_bfield_model(strand, B0=1*u.G, B1=10*u.G, ell=14*u.Mm):
    s = strand.field_aligned_coordinate
    L = strand.length
    return B0 + B1*(np.exp(-s/ell) + np.exp(-(L-s)/ell))

In [ ]:
date = '2020-01-01'
view = SkyCoord(lon=20*u.deg, lat=5*u.deg, radius=1*u.Rsun, obstime=date, frame='heliographic_stonyhurst')

In [ ]:
sdo = get_horizons_coord('SDO', time=date)

In [ ]:
arcade = semi_circular_bundle(100*u.Mm, 3*u.Mm, 10, inclination=5*u.deg, observer=view)
strands = [synthesizAR.Strand(f'strand{i}', s) for i,s in enumerate(arcade)]
for s in strands:
    s.field_strength = mikic_bfield_model(s, B1=25*u.G)

In [ ]:
skeleton = synthesizAR.Skeleton(strands)

In [ ]:
skeleton.peek(observer=sdo)

In [ ]:
skeleton.peek(
    observer=sdo,
    axes_limits=((-1000,1000)*u.arcsec, (-1000,1000)*u.arcsec)
)

In [ ]:
for s in skeleton.strands:
    plt.plot(s.field_aligned_coordinate_center_norm, s.field_strength_center, color='k', alpha=0.1)

## Configure HYDRAD Simulations

In [ ]:
hydrad_path = pathlib.Path('hydrad-clean')

In [ ]:
pydrad.configure.util.get_clean_hydrad(hydrad_path, from_github=True, overwrite=True)

In [ ]:
default_hydrad_config = pydrad.configure.data.get_defaults()
default_hydrad_config['general']['total_time'] = 13 * u.h
default_hydrad_config['general']['output_interval'] = 6 * u.s
default_hydrad_config['general']['logging_frequency'] = 10000
default_hydrad_config['general']['write_file_equation_terms'] = False
default_hydrad_config['general']['write_file_hydrogen_level_populations'] = False
default_hydrad_config['general']['write_file_timescales'] = False
default_hydrad_config['general']['write_file_physical'] = False
default_hydrad_config['general']['write_file_ion_populations'] = True
default_hydrad_config['general']['poly_fit_gravity'] = {
    'order': 6,
    'domains': np.linspace(0,1,5),
}
default_hydrad_config['general']['poly_fit_magnetic_field'] = {
    'order': 6,
    'domains': np.linspace(0,1,5),
}
# make any modifications to defaults that will apply to all strands here
default_hydrad_config['grid']['maximum_cell_width'] = 1 * u.Mm
# NOTE: Pinning this explicitly as using a low refinement level can result in a 
default_hydrad_config['grid']['maximum_cells'] = 30000
# NOTE: Setting this fairly low as we will use TRAC to deal with the refinement
default_hydrad_config['grid']['maximum_refinement_level'] = 4
default_hydrad_config['grid']['initial_refinement_level'] = 4
default_hydrad_config['grid']['linear_restriction'] = True
default_hydrad_config['grid']['enforce_conservation'] = False
default_hydrad_config['grid']['refine_on_hydrogen_energy'] = False
default_hydrad_config['grid']['adapt_every_n_time_steps'] = 10
default_hydrad_config['radiation']['decouple_ionization_state_solver'] = True
default_hydrad_config['radiation']['use_power_law_radiative_losses'] = True
default_hydrad_config['radiation']['elements_nonequilibrium'] = [
    'Fe',
	'Si',
	'Mg',
	'Ca',
	'O',
	'Ne',
]
default_hydrad_config['radiation']['rates_dataset'] = 'chianti_v10'
default_hydrad_config['initial_conditions']['footpoint_temperature'] = 1e4 * u.K
default_hydrad_config['solver']['minimum_temperature'] = 5e3 * u.K
default_hydrad_config['solver']['minimum_radiation_temperature'] = 1e4 * u.K
default_hydrad_config['solver']['safety_radiation'] = 0.01
default_hydrad_config['solver']['cutoff_temperature_fraction'] = 0.2

In [17]:
import astropy.units as u
foo = 9.91840105492 * u.Mm
print(f'{foo.to_value('cm'):1.4e}')

9.9184e+08


In [ ]:
class HeatingModel:

    def calculate_event_properties(self, config, loop):
        return config

In [ ]:
hydrad_interface = HYDRADInterface(
    'hydrad_results',
    base_config=default_hydrad_config,
    hydrad_dir=hydrad_path,
    use_gravity=True,
    use_magnetic_field=True,
    heating_model=HeatingModel(),
)

In [ ]:
skeleton.configure_loop_simulations(hydrad_interface)